### 북 리커맨드 레터 만들기



In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
import os

NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

def search_for_book_titles_naver(keyword):
    """
    This tool interacts with the Naver Book Search API to search for recent book titles related to a given keyword.

    Args:
        keyword (str): The keyword or phrase to search for in the books.

    Returns:
        list of str: A list of up to 5 book titles related to the keyword.
    """
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    }
    params = {
        "query": keyword,
        "display": 5,     # 최대 5개 결과
        "sort": "sim"    # 최신순 정렬
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # 오류 발생 시 예외

    items = response.json().get("items", [])
    title_list = [item["title"].replace("<b>", "").replace("</b>", "") for item in items]

    return title_list


In [20]:
import requests

def search_for_book_titles_google(keyword: str, max_results: int = 5) -> list[str]:
    """
    Uses Google Books API to search for book titles related to a given keyword.

    Args:
        keyword (str): The keyword to search for.
        max_results (int): The number of results to return (default 5).

    Returns:
        list of str: A list of book titles related to the keyword.
    """
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": keyword,
        "maxResults": max_results,
        "printType": "books",
        "langRestrict": "ko",  # 한글 도서 우선
    }

    response = requests.get(url, params=params)
    response.raise_for_status()

    items = response.json().get("items", [])
    titles = [item["volumeInfo"].get("title", "제목 없음") for item in items]

    return titles


In [48]:
def search_for_book_titles(keyword):
    # 1순위: Google Books
    titles = search_for_book_titles_google(keyword)

    # 만약 결과가 없다면 fallback Naver API로 검색
    if not titles:
        titles = search_for_book_titles_naver(keyword)

    return titles

In [47]:
keyword = "김금희"
result = search_for_book_titles(keyword)
print(result)

['대온실 수리 보고서 (김금희 장편소설)', '나의 폴라 일지(큰글자도서)', '대온실 수리 보고서 (큰글자도서) (김금희 장편소설)', '나의 폴라 일지', '敬愛の心']


In [30]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Data model
class BookLetterThemeOutput(BaseModel):
    """Output model for structured theme and sub-theme generation."""

    theme: str = Field(
        description="The main book letter theme based on the provided books titles."
    )
    sub_themes: list[str] = Field(
        description="List of sub-themes or key books items to investigate under the main theme, ensuring they are specific and researchable."
    )


# LLM with function call
structured_llm_book_letter = llm.with_structured_output(BookLetterThemeOutput)

# Prompt
system = """
You are an expert editorial assistant for a themed book letter. Your task is to create a consistent and compelling topic structure based on a given list of book titles.

Instructions:
- Carefully examine the book titles and infer the common thread or theme that connects them.
- Choose **one main theme** that reflects this connection. The theme should be written in the form of a specific, thought-provoking question in Korean.
- Then, write **five sub-themes**, each clearly and meaningfully derived from the main theme and the titles. These should be deeply tied to the books, not general topics.
- Your goal is to help the editor write a book letter that feels unified, insightful, and true to the books mentioned.

Do not invent unrelated or generic ideas.
Think like a book editor planning a well-curated issue around real books.

All output must be written in Korean.
"""

# This is the template that will feed into the structured LLM
theme_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Book titles: \n\n {book_titles}"),
    ]
)

# Chain together the system prompt and the structured output model
book_letter_generator = theme_prompt | structured_llm_book_letter

In [31]:
# test
output = book_letter_generator.invoke({"book_titles": result})
sub_themes = output.sub_themes
output

BookLetterThemeOutput(theme='온실의 보살핌과 마음의 기록이란 무엇인가?', sub_themes=['대온실에서의 인간관계와 감정의 교류가 만드는 이야기의 힘은?', '자연과의 교감을 통해 발견하는 자기 자신은 어떻게 드러나는가?', '일기라는 형식을 통해 우리는 어떻게 우리의 내면을 성찰할 수 있는가?', '대온실이라는 상징이 현대 사회에 던지는 질문은 무엇인가?', '사랑과 존경이 나와 타인을 연결하는 통로로서의 역할은 어떻게 표현되는가?'])

In [41]:
from typing import Dict, List, TypedDict, Annotated
# 딕셔너리 형태의 값들을 계속해서 누적해 나가기 위한 함수
# 왼, 오른 모두 Dict 인 경우, 이 둘을 함쳐서 하나의 Dict 으로 정의
def merge_dicts(left: Dict, right: Dict) -> Dict:
    return {**left, **right}


class State(TypedDict):
    keyword: str
    book_titles: List[str]
    book_letter_theme: BookLetterThemeOutput
    sub_theme_books: Dict[str, List[Dict]]
    results: Annotated[Dict[str, str], merge_dicts]
    messages: Annotated[List, add_messages]


In [54]:
def edit_book_letter(state: State) -> State:
    theme = state['book_letter_theme'].theme
    combined_book_letter = state['messages'][-1].content

    prompt = f"""
You are writing a book letter that should feel like a heartfelt personal message from someone close — such as a friend, sibling, or loved one.

Please rewrite the following book letter written on the theme: "{theme}"

{combined_book_letter}

Your revised version should:
- Feel like a handwritten note, not an editorial review.
- Be warm, reflective, and emotionally resonant.
- Sound as if you're genuinely recommending these books to someone you care about, not to the general public.
- Include a title that is written as a soft question, not a bold headline.
- If suggesting other books, limit them to literature, humanities, society, essay, economics, or science.

Use natural Korean language that feels familiar and comforting, like a message you'd send to someone you miss.

Avoid any formal, rigid, or mechanical language.

Output only the revised letter in Korean.
    """

    messages = [HumanMessage(content=prompt)]
    writer_llm = ChatOpenAI(model="gpt-4o-mini", temperature=1, max_tokens=8192)
    response = writer_llm.invoke(input=messages)

    return {"messages": [HumanMessage(content=f"Edited Book Letter:\n\n{response.content}")]}


In [55]:
import asyncio
from typing import Dict, List, TypedDict, Annotated
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
import operator
from tavily import TavilyClient, AsyncTavilyClient
import os


# 동기 함수들
# 주어진 키워드를 기반으로 관련된 책 제목을 뽑아주는 함수
def search_keyword(state: State) -> State:
    keyword = state['keyword']
    book_titles = search_for_book_titles(keyword)
    return {"book_titles": book_titles}


# book_titles 기반으로 북 레터의 대, 소주제를 정의
def generated_book_letter_theme(state: State) -> State:
    book_titles = state['book_titles']
    book_letter_theme = book_letter_generator.invoke({"book_titles": "\n".join(book_titles)})
    book_letter_theme.sub_themes = book_letter_theme.sub_themes[:5]
    return {"book_letter_theme": book_letter_theme}


# 각각의 세부 주제들에 대해서 비동기로 검색을 실행
# asyncio.gather 비동기로 한꺼번에 모아 실행
async def search_sub_themes(state: State) -> State:
    sub_themes = state['book_letter_theme'].sub_themes
    results = await asyncio.gather(*[search_sub_theme(sub_theme) for sub_theme in sub_themes])

    sub_theme_books = {}
    for result in results:
        sub_theme_books.update(result)

    return {"sub_theme_books": sub_theme_books}


# 비동기로 실행하는 Tavily 검색
# 세부 주제에 대해서 비동기로 북 검색
# sub_theme 에 대한 book letter info 저장함
async def search_sub_theme(sub_theme):
    async_tavily_client = AsyncTavilyClient()
    response = await async_tavily_client.search(
        query=sub_theme,
        max_results=3,
        topic="general",
        days=100,
        include_images=True,
        include_raw_content=True)
    images = response['images']
    results = response['results']

    book_letter_info = []
    for i, result in enumerate(results):
        book_letter_info.append({
            'title': result['title'],
            'image_url': images[i],
            'raw_content': result['raw_content'],
        })
    return {sub_theme: book_letter_info}

import aiohttp

async def search_sub_theme_google(sub_theme):
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": sub_theme,
        "maxResults": 3,
        "langRestrict": "ko",  # 한국어 도서 위주
        "printType": "books",
    }

    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            data = await resp.json()

    items = data.get("items", [])
    book_letter_info = []
    for item in items:
        info = item.get("volumeInfo", {})
        book_letter_info.append({
            "title": info.get("title", ""),
            "image_url": info.get("imageLinks", {}).get("thumbnail", ""),
            "raw_content": info.get("description", ""),
        })

    return {sub_theme: book_letter_info}


# 검색 후 세부 주제 본문 작성
def write_book_letter_section(state: State, sub_theme: str) -> Dict:
    return asyncio.run(write_book_letter_section_async(state, sub_theme))


async def write_book_letter_section_async(state: State, sub_theme: str) -> Dict:
    books = state['sub_theme_books'][sub_theme]

    #Prepare a list of formatted book references including title, image, and a snippet of raw content
    books_references = "\n".join(
        [f"Title: {book['title']}\nContent: {book['raw_content']}..." for book in books]
    )

    prompt = f"""
    Write a book letter section for the sub-theme: "{sub_theme}".

    Use the following books as reference and include relevant points from both their titles, images, and content:
    <book>
    {books_references}
    </book>
    Summarize the key points and trends related to this sub-theme, and ensure you reference the images where they add value to the discussion.
    Keep the tone engaging and informative for book letter readers. You should write in Korean
    """

    messages = [HumanMessage(content=prompt)]
    response = await llm.ainvoke(messages)
    return {"results": {sub_theme: response.content}}


# 세부 주제들에 대해서 결합
def aggregate_results(state: State) -> State:
    theme = state['book_letter_theme'].theme
    combined_book_letter = f"# {theme}\n\n"
    for sub_theme, content in state['results'].items():
        combined_book_letter += f"# {sub_theme}\n{content}\n"
    return {"messages": [HumanMessage(content=f"Generated Book Letter:\n\n{combined_book_letter}")]}


# LangGraph 구성
workflow = StateGraph(State)
workflow.add_node("editor", edit_book_letter)
workflow.add_node("search_keyword", search_keyword)
workflow.add_node("generated_theme", generated_book_letter_theme)
workflow.add_node("search_sub_themes", search_sub_themes)
workflow.add_node("aggregate", aggregate_results)

for i in range(5):
    node_name = f"write_section_{i}"
    workflow.add_node(node_name, lambda s, i=i: write_book_letter_section(s, s['book_letter_theme'].sub_themes[i]))

# 엣지 연결
workflow.add_edge(START, "search_keyword")
workflow.add_edge("search_keyword", "generated_theme")
workflow.add_edge("generated_theme", "search_sub_themes")
for i in range(5):
    workflow.add_edge("search_sub_themes", f"write_section_{i}")
    workflow.add_edge(f"write_section_{i}", "aggregate")
workflow.add_edge("aggregate", "editor")
workflow.add_edge("editor", END)

graph = workflow.compile()


In [57]:
keyword = input("Enter a keyword for the book letter")

inputs = {"keyword": keyword}
async for output in graph.astream(inputs, stream_mode="updates"):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(print(value))
    print("\n---\n")

Output from node 'search_keyword':
---
{'book_titles': ['나는 그것에 대해 아주 오랫동안 생각해', '너무 한낮의 연애', '복자에게', '오직 한 사람의 차지', '기획회의 536호']}
None

---

Output from node 'generated_theme':
---
{'book_letter_theme': BookLetterThemeOutput(theme='사랑이란 무엇인가?', sub_themes=["사랑의 고민과 갈등: '나는 그것에 대해 아주 오랫동안 생각해'에서의 내면적 탐구", "낮은 조명의 연애: '너무 한낮의 연애'가 보여주는 사랑의 현실", "복잡한 관계의 재정의: '복자에게'를 통해 본 인연의 의미", "독점의 사랑: '오직 한 사람의 차지'에서의 소유와 헌신의 갈림길", "창작과 현실의 경계: '기획회의 536호'가 제시하는 사랑의 진정한 모습"])}
None

---

Output from node 'search_sub_themes':
---
{'sub_theme_books': {"사랑의 고민과 갈등: '나는 그것에 대해 아주 오랫동안 생각해'에서의 내면적 탐구": [{'title': '외딴방 (한국문학전집 009)', 'image_url': 'http://books.google.com/books/content?id=gq5HBQAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api', 'raw_content': '문학동네 한국문학전집 제9권 신경숙 장편소설 『외딴방』(1995)은 80년대의 암흑기 속에서 문학에의 꿈을 키워나가던 신경숙의 시원(始原)을 만날 수 있는 자전적 성장소설로, 현재진행형의 글쓰기를 통해 오로지 문학만이 보여줄 수 있는 깊이와 아름다움을 표현해내어 독자와 언론의 열렬한 관심은 물론 문단의 다양한 진영에서 일치된 찬사를 이끌어냈다. 내용과 형식 양면에서 새로운 리얼리즘의 가능성을 열어 보인 『외딴방』은 

CancelledError: 

In [ ]:
## test
import nest_asyncio
nest_asyncio.apply()

asyncio.run(search_sub_theme("사회적 문화에서의 눈물의 역할: 감정 표현의 다양성과 그 수용 양상"))

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
test_state = {
    'sub_theme_books': {
        '사회적 문화에서의 눈물의 역할: 감정 표현의 다양성과 그 수용 양상': [
            {
                'title': '눈물의 의미: 감정의 표현과 신체적 반응 - blogmina.com',
                'raw_content': '''간단설명\체적 반응을 넘어서, 감정의 깊이를 나타내고, 사회적 관계를 더욱 강화하는 중요한 요소입니다. 눈물은 다양한 원인에 의해 발생하며, 그 목적은 눈을 보호하고, 감정을 해소하며, 사회적 상호작용을 돕는 것입니다. 감정적으로나 신체적으로 어려움을 겪을 때 흐르는 눈물은 우리에게 중요한 메시지를 전달하는 신호일 수 있으며, 이를 이해하는 것은 더 나은 사회적 관계를 만드는 데 기여할 수 있습니다.\n\n'''
            },
            {
                'title': '눈물의 역사와 문화적 의미 : 네이버 ... - 네이버 블로그',
                'raw_content': '''\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n프롤로그블로그 | 지도서재안부\n블로그\n\n\n\n |  | \n | 전체보기574개의 글전체보기목록열기 | \n전체보기 574개의 글\n\n |  | \n, 다른 문화에서는 눈물을 힘과 인간성의 표시로 여깁니다.\n\n\u200b\n\n눈물의 치유력\n\n\u200b\n\n최근 연구에 따르면 눈물에는 치유력이 있는 것으로 나타났습니다. 눈물에는 스트레스 호르몬인 코르티솔이 포함되어 있으며, 눈물을 흘리면 스트레스와 긴장을 줄이는 데 도움이 될 수 있습니다. 또한, 눈물에는 항균 성분이 포함되어 있으며, 감염으로부터 눈을 보호하는 데 도움이 될 수 있습니다.\n\n\u200b\n\n결론\n\n\u200b\n\n눈물은 인간 경험의 필수적인 부분입니다. 그것은 감정을 표현하고, 소통하며, 치유하는 수단으로 오랜 역사와 문화적 의미를 지니고 있습니다. 눈물에 대한 우리의 이해는 우리가 다른 문화를 인정하고, 인간의 다양성을 축하하는 데 도움이 될 수 있습니다.\n\n'''
            }
        ]
    },
    'results': {}
}

## test

asyncio.run(write_book_letter_section_async(test_state, '사회적 문화에서의 눈물의 역할: 감정 표현의 다양성과 그 수용 양상'))